# Imports

In [ ]:
import os
import tensorflow
import numpy as np
import pandas as pd

from scipy.signal import resample

from random import sample

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, LeakyReLU, Input, Add, GlobalAveragePooling1D

# Verify GPU
A simple check to see if we use the GPU or not.

In [ ]:
if tensorflow.config.list_physical_devices('GPU'):
    print("GPU device found, using MPS for acceleration.")
else:
    print("No GPU device found, falling back to CPU.")

# Functions
## map_labels
This function can be used to simplify the labels. With a given label dictionary, we can map the labels to a less specific label.

In [ ]:
def map_labels(data, label_dict, label_mapping):
    """
    Maps annotations in a dataset to their corresponding labels using a given mapping.

    Args:
        data (pd.DataFrame): A DataFrame containing an 'annotation' column with values to be mapped.
        label_dict (dict): A dictionary containing mappings of annotation labels.
                                Keys are label mapping names, and values are Pandas Series or DataFrames
                                where the index represents annotation keys and values represent labels.
        label_mapping (str): The key in `anno_label_dict` corresponding to the desired label mapping.

    Returns:
        pd.DataFrame: The input DataFrame with an added or updated 'label' column containing the mapped labels.
    """
    data['label'] = (label_dict[label_mapping].reindex(data['annotation']).to_numpy())
    return data

## normalize
Normalize the data to make it better for generalization

In [ ]:
def normalize(data, feature_cols):
    """
    Normalizes the specified feature columns of a DataFrame using Min-Max scaling.

    Args:
        data (pd.DataFrame): A DataFrame containing the features to be normalized.
        feature_cols (list of str): List of column names in `data` to normalize.
        
    Returns:
        pd.DataFrame: A new DataFrame with the specified columns normalized to the range [0, 1].
        The original `data` DataFrame is modified in place.
        
    Notes:
        - Min-Max normalization scales each feature column to the range [0, 1] based on 
          the minimum and maximum values of the column.
        - This transformation is often used to prepare data for machine learning models 
          that are sensitive to feature magnitudes.
    """
    scaler = MinMaxScaler()
    data[feature_cols] = scaler.fit_transform(data[feature_cols])
    return data

## extract_windows
A function to create windows. We can also change the frequency of the windows by downsampling.

In [ ]:
def extract_windows(data, winsize=90, input_frequency=100, output_frequency=64):
    """
    Extracts sliding windows from time series data and labels for classification tasks.

    Args:
        data (pd.DataFrame): Time series DataFrame with 'x', 'y', 'z' (accelerometer data) and 'label'.
                             Index should be a datetime-like index.
        winsize (int): Window size in seconds.
        input_frequency (int): Sampling frequency of input data in Hz.
        output_frequency (int): Desired output frequency in Hz (must be a divisor of input_frequency).

    Returns:
        X (np.ndarray): Shape (n_samples, output_samples, 3), accelerometer windows.
        Y (np.ndarray): Shape (n_samples,), most frequent label per window.
    """
    # Calculate window size in samples and target output samples
    window_samples = winsize * input_frequency
    output_samples = winsize * output_frequency  # Expected downsampled length

    X, Y = [], []

    # Sliding window extraction
    for start in range(0, len(data) - window_samples + 1, window_samples):
        window = data.iloc[start:start + window_samples]

        # Skip if missing values exist
        if window.isna().any().any() or len(window) != window_samples:
            continue

        # Extract and resample accelerometer data
        x = window[['x', 'y', 'z']].to_numpy()
        x = resample(x, output_samples)  # Resample to match output frequency

        # Extract the most frequent label (mode)
        y = window['label'].mode().iloc[0]

        X.append(x)
        Y.append(y)

    X = np.stack(X) if X else np.empty((0, output_samples, 3))
    Y = np.array(Y) if Y else np.empty((0,))

    return X, Y

## residual_block
A function to create the residual blocks inside the model.

In [ ]:
def residual_block(inputs, filters, kernel_size=3, dropout_rate=0.5):
    """
    Residual block with Conv1D layers, BatchNormalization, and LeakyReLU activations.
    Includes a shortcut connection.
    """
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)
    x = Dropout(dropout_rate)(x)
    
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    
    shortcut = inputs
    if inputs.shape[-1] != filters:
        shortcut = Conv1D(filters=filters, kernel_size=1, padding='same', kernel_initializer='he_normal')(inputs)
    
    x = Add()([x, shortcut])
    x = LeakyReLU(negative_slope=0.1)(x)
    return x

## create_model
A function to create the model.

In [ ]:
def create_model(
    num_classes, 
    input_shape, 
    dropout_rate=0.5,
    filters=128, 
    kernel_size=5, 
    residual_blocks=3,
    learning_rate=3e-4
):
    inputs = Input(shape=input_shape)
    
    # Initial Convolution
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)
    x = MaxPooling1D(pool_size=3)(x)
    
    # First residual block set
    for _ in range(residual_blocks):
        x = residual_block(x, filters=filters)
    x = MaxPooling1D(pool_size=3)(x)
    
    # Second residual block set
    x = Conv1D(filters=filters * 2, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)
    for _ in range(3):
        x = residual_block(x, filters=filters * 2)
    x = MaxPooling1D(pool_size=5)(x)

    # Third residual block set
    for _ in range(residual_blocks):
        x = residual_block(x, filters=filters * 2)
    x = MaxPooling1D(pool_size=5)(x)

    # Fourth residual block set (512 channels)
    x = Conv1D(filters=filters * 4, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(negative_slope=0.1)(x)
    for _ in range(3):
        x = residual_block(x, filters=filters * 4)
    x = MaxPooling1D(pool_size=5)(x)
    
    # Fully Connected Layer
    x = Dense(filters * 8, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(dropout_rate)(x)

    # Final Layer
    x = GlobalAveragePooling1D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    # Create and Compile Model
    model = Model(inputs=inputs, outputs=x)
    model.compile(
        optimizer=Adam(learning_rate=learning_rate), 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

# Print input files

In [ ]:
# Dictionary of labels
label_dict = pd.read_csv('/kaggle/input/capture-24-human-activity-recognition/capture24/annotation-label-dictionary.csv', index_col='annotation', dtype='string')
print("Annotation-Label Dictionary")
print(label_dict)

# Chosen label mapping
label_mapping = "label:Willetts2018"
print(label_dict[label_mapping])
print()

# Print files
print('Content of data/')
print(sorted(os.listdir('/kaggle/input/capture-24-human-activity-recognition/capture24')))
print()

# All of the labels
all_labels = list({*label_dict[label_mapping]})
print('All of the possible labels:')
print(all_labels)

# Training the model
## Training parameters

In [ ]:
# Path of the dataset
path = '/kaggle/input/capture-24-human-activity-recognition/capture24/'

# The files in the dataset
file_list = sorted([f for f in os.listdir(path) if f.endswith('.csv') and f.startswith('P')])

# Stopping criteria for testing
counter = 0
limit = 200
limit = min(limit, len(file_list))
print("Number of files used: ", limit)

# Initialize label encoder
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Model initialized flag
model_initialized = False

# Number of files used purely for testing
num_test_files = 25
print("Number of test files: ", num_test_files)

# Randomly select test files from the file list
test_files = sample(file_list, num_test_files)

# Remaining files used for training
train_files = [f for f in file_list if f not in test_files]

# Number of chunks to split the data into
num_chunks = 4
num_chunks = min(num_chunks, limit)
print("Number of chunks used in training: ", num_chunks)

# Size of each chunk
chunk_size = len(train_files) // num_chunks
# Remaining files that don't fit evenly into a chunk
remaining_files = len(train_files) % num_chunks

# Distribute remaining files across the chunks
chunks = []
for i in range(num_chunks):
    # Give the extra files to the first few chunks
    chunk = train_files[i * chunk_size: (i + 1) * chunk_size]
    if i < remaining_files:
        chunk.append(train_files[(num_chunks * chunk_size) + i])
    chunks.append(chunk)

## Training on the chunks

In [ ]:
# Iterate over the chunks
for chunk_idx, chunk in enumerate(chunks):
    print(f"Processing chunk {chunk_idx + 1}/{len(chunks)}")

    data_list = []
    # Iterate over the files in the chunk
    for file in chunk:
        if counter >= limit:
            break

        counter += 1
        file_path = os.path.join(path, file)
        file_path = os.path.join(file_path, file)
        data = pd.read_csv(file_path, index_col='time', parse_dates=['time'],
                           dtype={'x': 'f4', 'y': 'f4', 'z': 'f4', 'annotation': 'string'})
        
        data = map_labels(data, label_dict, label_mapping)      
        data = normalize(data, ['x','y','z'])
        data_list.append(data)

    X_chunk, Y_chunk = [], []

    # Extract the windows from the data
    for data in data_list:
        X_, Y_ = extract_windows(data)
        if X_ is None or Y_ is None or len(X_) == 0 or len(Y_) == 0:
            print(f"Warning: Empty window extraction in chunk {chunk_idx}.")
        else:
            X_chunk.append(X_)
            Y_chunk.append(Y_)

    # A skip if the chunk is empty
    if len(X_chunk) == 0 or len(Y_chunk) == 0:
        print(f"Error: No valid windows in chunk {chunk_idx}. Skipping...")
        continue

    X_chunk = np.concatenate(X_chunk, axis=0)
    Y_chunk = np.concatenate(Y_chunk, axis=0)

    # If the model is not initialize it
    if not model_initialized:
        input_shape = (X_chunk.shape[1], X_chunk.shape[2])
        print(f"The input shape is: {input_shape}")
        print(f"Number of windows: {X_chunk.shape[0]}")
        
        model = create_model(num_classes=len(all_labels), input_shape=input_shape)
        model_initialized = True

    # Encode the labels
    Y_chunk_encoded = label_encoder.transform(Y_chunk)

    # Perform stratified train-validation split (90% train, 10% validation)
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_chunk, Y_chunk_encoded, test_size=0.1, stratify=Y_chunk_encoded, random_state=42
    )

    # Fit the model with the labels
    history = model.fit(
        X_train, Y_train,
        validation_data=(X_val, Y_val),
        epochs=40,
        batch_size=16,
        verbose=0,
    )

    print(f"Validation loss: {min(history.history['val_loss']):.4f}")
    print(f"Validation accuracy: {max(history.history['val_accuracy']):.4f}")

# Testing the model

In [ ]:
data_list = []

# Iterate over the test files
for test_idx, test_file in enumerate(test_files):
    print(f"Processing file {test_idx + 1}/{len(test_files)}")

    file_path = os.path.join(path, file)
    file_path = os.path.join(file_path, file)
    data = pd.read_csv(
        file_path, index_col='time', parse_dates=['time'],
        dtype={'x': 'f4', 'y': 'f4', 'z': 'f4', 'annotation': 'string'}
    )

    data = map_labels(data, label_dict, label_mapping)
    data = normalize(data, ['x', 'y', 'z'])
    data_list.append(data)

X_test, Y_test = [], []

# Extract the windows
for data in data_list:
    X_, Y_ = extract_windows(data)
    if X_ is None or Y_ is None or len(X_) == 0 or len(Y_) == 0:
        print(f"Warning: Empty window extraction in test.")
    else:
        X_test.append(X_)
        Y_test.append(Y_)

X_test = np.concatenate(X_test, axis=0)
Y_test = np.concatenate(Y_test, axis=0)

# Encode the labels
Y_test_encoded = label_encoder.transform(Y_test)

# Make predictions
Y_pred = model.predict(X_test)
Y_pred = np.argmax(Y_pred, axis=1)

# Decode predictions
Y_pred = label_encoder.inverse_transform(Y_pred)

# Subset labels present in the test set
present_labels = np.unique(Y_test)
present_labels_encoded = label_encoder.transform(present_labels)

# Calculate metrics
acc = accuracy_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred, average='weighted', labels=present_labels)
recall = recall_score(Y_test, Y_pred, average='weighted', labels=present_labels)
precision = precision_score(Y_test, Y_pred, average='weighted', labels=present_labels)

# Confusion matrix for present labels
cm = confusion_matrix(Y_test, Y_pred, labels=present_labels, normalize='true')

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=present_labels)
disp.plot(cmap='Blues', values_format=".2%")
disp.ax_.set_title("Confusion Matrix (Percentage)")

# Print metrics
print(f"  Accuracy: {acc:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Confusion Matrix:\n{cm}")

## Save the model

In [ ]:
# Save the model
model.save("/kaggle/working/CNN.keras")